In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb 
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [2]:
df = pd.read_csv('data/xgboost_tutorial_data.csv')

In [5]:
y = df['woba']
X = df[[
    'pa',
    'home_run',
    'k_percent',
    'bb_percent',
    'sweet_spot_percent',
    'avg_best_speed',
    'swing_percent'
]]

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size = 0.2,
    random_state = 2002,
    stratify = None
) # train test split
print(f"Number of training set players: {X_train.shape[0]}")
print(f"Number of test set players: {X_test.shape[0]}")

Number of training set players: 264
Number of test set players: 66


In [6]:
model = xgb.XGBRegressor(
    objective='reg:squarederror',
    random_state=2002,
    eval_metric='rmse'
)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)


In [9]:
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
print("Model Performance Metrics")
print(f"Variance explained by the model: {r2:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAE:{mae:.4f}")

Model Performance Metrics
Variance explained by the model: 0.5424
RMSE: 0.0237
MAE:0.0195
